In [15]:
import numpy as np
from Common_function import *
import matplotlib.pyplot as plt
from itertools import product
import itertools

x = 3  #how are is the distance set
Total_E_UGV = 500000
Total_E_UAV = 15000
UAV_E_cost = 1250

UGV_E_cost_with_UAV = 1000
UGV_E_cost_without_UAV = 800
Charging_speed = 1000
speed_uav =30
speed_ugv =10
ten_minets = 2
survey_time = ten_minets/6
UAV_E_s_cost = 2500*ten_minets
print(UAV_E_s_cost)
# waypoints = [(20, 10), (24, 37), (36, 43), (8, 8), (0, 50), (30, 44), (28, 4), (32, 25)]
# # waypoints = [(20,10), (10,10),(15,30)]
# # plan_output = "0 -> 1 -> 2 -> 0"
# plan_output = "0 -> 3 -> 4 -> 1 -> 5 -> 2 -> 7 -> 6 -> 0"
waypoints = [(20,10), (10,10),(15,30)]
plan_output = "0 -> 1 -> 2 -> 0"
max_radius = ((Total_E_UAV - UAV_E_s_cost) / UAV_E_cost) / 2
radii = np.linspace(0.1, max_radius, x)
ordered_points = [waypoints[int(node)] for node in plan_output.split() if node.isdigit()]
ordered_points.append(ordered_points[0])


5000


In [16]:
def calculate_UAV_inter_distances_inter(UAV_path):
    UAV_inter_distances = 0
    group_distance=0
    for i in range(len(UAV_path) - 1):
        group_distance += compute_distance(UAV_path[i], UAV_path[i + 1])
    UAV_inter_distances=group_distance
    return UAV_inter_distances

In [17]:
import numpy as np

# I am assuming the functions like angle_between_points, point_on_circle, find_meeting_point_with_survey_final, etc. 
# are previously defined. However, as they are not provided here, I'll make the necessary modifications based on the code provided.

def compute_optimized_paths_for_radius_updated_v5(ordered_points, radius, speed_ugv, speed_uav, survey_time):
    UGV_outer_path = []
    UGV_path = [ordered_points[0]]
    UAV_path = []
    UGVD_inter_without_drone = []
    UGVD_inter_with_drone = []
    chord_end = None
    prev_chord_end = None
    
    radius_set = []
    
    for i, point in enumerate(ordered_points[:-1]):
        # Replace current_radius with radius
        if 0 < i < len(ordered_points) - 2:
            # assuming angle_between_points and point_on_circle functions are defined previously
            angle = angle_between_points(ordered_points[i-1], ordered_points[i])
            nextangle = angle_between_points(ordered_points[i], ordered_points[i+1])
            chord_start = point_on_circle(ordered_points[i], angle - np.pi, radius)
            chord_end = point_on_circle(ordered_points[i], nextangle, radius)
            radius_set.append(radius)
            
            if prev_chord_end and i > 1 and circles_overlap(ordered_points[i], radius, ordered_points[i-1], radius):
                chord_start = prev_chord_end

            # assuming find_meeting_point_with_survey_final function is defined previously
            final_meeting_point, final_wait_time = find_meeting_point_with_survey_final(
                ordered_points[i], radius, chord_start, chord_end, 
                speed_ugv, speed_uav, survey_time
            )
     
            
            if chord_start:
                UAV_path_segment = [chord_start, point, final_meeting_point]
                UAV_path.append(UAV_path_segment)
                UGV_path.append(chord_start)
                # assuming calculate_UAV_inter_distances_inter function is defined previously
                UAV_path_segment_distance = calculate_UAV_inter_distances_inter(UAV_path_segment)
                if i == 1:
                    UGV_outer_path.append([ordered_points[0], chord_start])
                else:
                    UGV_outer_path.append([prev_chord_end, chord_start])
                    
            if chord_end:
                UGV_path.append(chord_end)
            # assuming compute_distance function is defined previously
            inter_distance_without_drone = compute_distance(chord_start, final_meeting_point)
            inter_distance_with_drone = compute_distance(final_meeting_point, chord_end)
            UGVD_inter_without_drone.append(inter_distance_without_drone)
            UGVD_inter_with_drone.append(inter_distance_with_drone)
            prev_chord_end = chord_end

    UGV_path.append(ordered_points[-1])
    UGV_outer_path.append([chord_end,ordered_points[-1]])
    
    return radius_set, UGV_path, UAV_path, UGV_outer_path, UGVD_inter_without_drone, UGVD_inter_with_drone, final_wait_time



In [18]:
def compute_path_for_one_point(ordered_points, point_index, radius, speed_ugv, speed_uav, survey_time):
    if point_index <= 0 or point_index >= len(ordered_points) - 1:
        raise ValueError("Invalid point index. It should be between 1 and len(ordered_points) - 2.")
    
    UGV_outer_path = []
    UGV_path = [ordered_points[0]]
    UAV_path = []
    UGVD_inter_without_drone = []
    UGVD_inter_with_drone = []
    chord_end = None
    prev_chord_end = None
    final_wait_set = []
    
    
    point = ordered_points[point_index]
    
    # Assuming angle_between_points and point_on_circle functions are defined previously
    angle = angle_between_points(ordered_points[point_index - 1], ordered_points[point_index])
    nextangle = angle_between_points(ordered_points[point_index], ordered_points[point_index + 1])
    chord_start = point_on_circle(ordered_points[point_index], angle - np.pi, radius)
    chord_end = point_on_circle(ordered_points[point_index], nextangle, radius)
    
    
    if point_index > 1 and circles_overlap(ordered_points[point_index], radius, ordered_points[point_index - 1], radius):
        chord_start = prev_chord_end

    # Assuming find_meeting_point_with_survey_final function is defined previously
    final_meeting_point, final_wait_time = find_meeting_point_with_survey_final(
        ordered_points[point_index], radius, chord_start, chord_end, 
        speed_ugv, speed_uav, survey_time
    )
    final_wait_set.append(final_wait_time)
    
    if chord_start:
        UAV_path_segment = [chord_start, point, final_meeting_point]
        UAV_path.append(UAV_path_segment)
        UGV_path.append(chord_start)

        # Assuming calculate_UAV_inter_distances_inter function is defined previously
        UAV_path_segment_distance = calculate_UAV_inter_distances_inter(UAV_path_segment)
        if point_index == 1:
            UGV_outer_path.append([ordered_points[0], chord_start])
        else:
            UGV_outer_path.append([prev_chord_end, chord_start])
                    
    if chord_end:
        UGV_path.append(chord_end)

    # Assuming compute_distance function is defined previously
    inter_distance_without_drone = compute_distance(chord_start, final_meeting_point)
    inter_distance_with_drone = compute_distance(final_meeting_point, chord_end)
    UGVD_inter_without_drone.append(inter_distance_without_drone)
    UGVD_inter_with_drone.append(inter_distance_with_drone)
    
    return radius, UGV_path, UAV_path, UGV_outer_path, UGVD_inter_without_drone, UGVD_inter_with_drone, final_wait_set



In [19]:
def calculate_total_wait_time(final_wait_time):
    return sum(final_wait_time)
def calculate_UGV_total_distance(UGV_path, compute_distance):
    UGV_total_distance = 0
    for i in range(len(UGV_path) - 1):
        UGV_total_distance += compute_distance(UGV_path[i], UGV_path[i + 1])
    return UGV_total_distance

def calculate_UGV_outer_path_distances(UGV_outer_path):
    UGV_outer_path_distances = []
    for group in UGV_outer_path:
        UGV_outer_path_distances.append(compute_distance(group[0], group[1]))
    return UGV_outer_path_distances

def calculate_UAV_inter_distances(UAV_path):
    UAV_inter_distances = []
    for group in UAV_path:
        group_distance = 0
        for i in range(len(group) - 1):
            group_distance += compute_distance(group[i], group[i + 1])
        UAV_inter_distances.append(group_distance)
    return format_list_to_decimal(UAV_inter_distances)


In [20]:

UGVdistances = []
UAV_total_distances = []
UAV_distances_set = []
ordered_points = compute_ordered_points(plan_output, waypoints)
result = []

In [21]:
radius, UGV_path, UAV_path, UGV_outer_path, UGVD_inter_without_drone, UGVD_inter_with_drone, final_wait_set = compute_path_for_one_point(ordered_points, 2, 5, speed_ugv, speed_uav, survey_time)

In [22]:
UGV_outer_path

[[None, (13.787321874818335, 25.149287499273342)]]

In [23]:
total_wait_time = calculate_total_wait_time(final_wait_set)
UGV_total_distance = calculate_UGV_total_distance(UGV_path, compute_distance)
UGV_outer_path_distances = calculate_UGV_outer_path_distances(UGV_outer_path)

UGV_outer_path_E_cost = [distance * UGV_E_cost_with_UAV for distance in UGV_outer_path_distances]
total_time = total_wait_time + UGV_total_distance / speed_ugv


UAV_inter_distances = calculate_UAV_inter_distances(UAV_path)
UAV_inter_distances = [float(distance) for distance in UAV_inter_distances]
UAV_inter_E_cost = [(distance * UAV_E_cost) + UAV_E_s_cost for distance in UAV_inter_distances]
UAV_distances_set = calculate_UAV_distances(UAV_path, compute_distance)

UGV_energy_remaining, UAV_energy_remaining = calculate_energy_remaining(Total_E_UGV, Total_E_UAV, UGV_outer_path_distances, UGVD_inter_with_drone, UAV_distances_set, UAV_path, UAV_E_cost, UAV_E_s_cost, UGVD_inter_without_drone, UGV_E_cost_without_UAV, Charging_speed)

# print(f"radius_combination:{radius_combination}")
if UGV_energy_remaining == -1 or UAV_energy_remaining == -1:
    
    result.append((radius_set,None))
else:
    result.append((radius_set,round(total_time,2)))

Error with points: None and (13.787321874818335, 25.149287499273342)


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
UGV_path

In [ ]:
UAV_path

In [ ]:
UGV_outer_path

In [ ]:
UGVD_inter_without_drone

In [ ]:
UGVD_inter_with_drone